# Chapter 5 - Exercise 5: Perfusion System Design & Analysis
## Biofabrication Course - VU Brussels

**Learning Objectives:**
- Design perfusion flow rates for different tissue constructs
- Analyze oxygen and glucose concentration gradients
- Calculate shear stress in perfusion systems
- Balance nutrient delivery with mechanical stress
- Optimize perfusion parameters for long-term culture

**Instructions:**
1. Read through Sections 1-11 to understand the baseline system
2. Complete the 10 TASKS in Section 12 by modifying parameters in Section 2
3. Re-run the entire notebook after each parameter change
4. Record your observations and answers

**Key Concepts:**
- Convective mass transport vs. diffusion
- Damköhler number (reaction/diffusion ratio)
- Péclet number (convection/diffusion ratio)
- Wall shear stress calculations
- Multi-constraint optimization

---

In [ ]:
# ============================================================================# CHAPTER 5 - EXERCISE 5: PERFUSION SYSTEM DESIGN & ANALYSIS# Course: Biofabrication - VU Brussels# Topic: Perfusion Bioreactor Design for Tissue Engineering# ============================================================================"""LEARNING OBJECTIVES:- Design perfusion flow rates for different tissue constructs- Analyze oxygen and glucose concentration gradients- Calculate shear stress in perfusion systems- Balance nutrient delivery with mechanical stress- Optimize perfusion parameters for long-term cultureWHAT YOU'LL LEARN:- How perfusion overcomes diffusion limitations- The trade-off between nutrient delivery and shear stress- How to calculate concentration gradients in flowing systems- The importance of residence time and flow velocity- How different tissues require different perfusion strategiesINSTRUCTIONS:1. Read through the entire program to understand the calculations2. Complete the TASKS at the end by modifying parameters in Section 23. Re-run the program after each parameter change4. Record your observations and answersKEY EQUATIONS USED:1. Oxygen consumption: dC/dt = -k·C (first-order kinetics)2. Convection-diffusion: ∂C/∂t = D·∂²C/∂x² - v·∂C/∂x - k·C3. Wall shear stress: τ = (6·μ·Q)/(w·h²) for rectangular channels4. Residence time: t_res = V/Q5. Damköhler number: Da = (k·L²)/D"""import numpy as npimport matplotlib.pyplot as pltfrom scipy.integrate import odeint# ============================================================================# SECTION 1: CELL TYPE DATABASE# ============================================================================print("="*80)print("CHAPTER 5 - EXERCISE 5: PERFUSION SYSTEM DESIGN & ANALYSIS")print("="*80)# Database of cell metabolic propertiesCELL_TYPES = {    'Hepatocytes': {        'O2_consumption': 0.40,      # nmol/s per million cells        'glucose_consumption': 0.50,  # nmol/s per million cells        'lactate_production': 0.60,   # nmol/s per million cells        'max_shear_stress': 0.5,      # Pa (dynes/cm²)        'optimal_density': 10e6,      # cells/cm³        'description': 'High metabolism, moderate shear tolerance'    },    'Cardiomyocytes': {        'O2_consumption': 0.80,        'glucose_consumption': 1.00,        'lactate_production': 1.20,        'max_shear_stress': 1.0,        'optimal_density': 50e6,        'description': 'Very high metabolism, needs pulsatile flow'    },    'Chondrocytes': {        'O2_consumption': 0.02,        'glucose_consumption': 0.05,        'lactate_production': 0.08,        'max_shear_stress': 0.1,        'optimal_density': 5e6,        'description': 'Low metabolism, very shear-sensitive'    },    'Osteoblasts': {        'O2_consumption': 0.15,        'glucose_consumption': 0.20,        'lactate_production': 0.25,        'max_shear_stress': 2.0,        'optimal_density': 20e6,        'description': 'Moderate metabolism, benefits from shear'    },    'Endothelial Cells': {        'O2_consumption': 0.10,        'glucose_consumption': 0.15,        'lactate_production': 0.18,        'max_shear_stress': 5.0,        'optimal_density': 8e6,        'description': 'Require physiological shear stress (1-3 Pa)'    },    'MSCs': {        'O2_consumption': 0.05,        'glucose_consumption': 0.08,        'lactate_production': 0.10,        'max_shear_stress': 0.3,        'optimal_density': 3e6,        'description': 'Low metabolism, moderate shear tolerance'    }}print("\n📊 AVAILABLE CELL TYPES FOR PERFUSION CULTURE:")print("-" * 80)print(f"{'Cell Type':<20} {'O₂ Rate':<12} {'Glucose':<12} {'Max Shear':<12}")print(f"{'':20} {'nmol/s/10⁶':<12} {'nmol/s/10⁶':<12} {'(Pa)':<12}")print("-" * 80)for cell_name, props in CELL_TYPES.items():    print(f"{cell_name:<20} {props['O2_consumption']:<12.2f} "          f"{props['glucose_consumption']:<12.2f} {props['max_shear_stress']:<12.2f}")# ============================================================================# SECTION 2: PERFUSION SYSTEM PARAMETERS (MODIFY THESE FOR TASKS)# ============================================================================print("\n" + "="*80)print("SECTION 2: PERFUSION SYSTEM CONFIGURATION")print("="*80)# ⚙️ MODIFY THESE PARAMETERS TO COMPLETE THE TASKS BELOW ⚙️# Construct geometryscaffold_length = 1.0           # cm (construct length in flow direction)scaffold_width = 1.0            # cmscaffold_height = 0.5           # cmscaffold_porosity = 0.70        # 70% void space for flowscaffold_tortuosity = 2.0       # path tortuosity factor# Cell parameterscell_type = 'Hepatocytes'       # Choose from: 'Hepatocytes', 'Cardiomyocytes',                                 # 'Chondrocytes', 'Osteoblasts',                                 # 'Endothelial Cells', 'MSCs'cell_density = 10e6             # cells/cm³ of scaffold# Flow parametersflow_rate = 100                 # μL/min (microliters per minute)channel_height = 100e-4         # cm (100 μm - microchannel height)# Medium composition (typical values)inlet_O2_concentration = 200    # μM (air-saturated medium at 37°C)inlet_glucose_concentration = 5000  # μM (5 mM - typical culture medium)inlet_lactate_concentration = 0     # μM (fresh medium)# Critical thresholds for cell survivalcritical_O2 = 40               # μM (below this: hypoxia)critical_glucose = 1000        # μM (below this: starvation)critical_lactate = 10000       # μM (above this: toxicity)# Physical constantsmedium_viscosity = 0.001       # Pa·s (similar to water)O2_diffusion_coeff = 2.5e-5    # cm²/s (in aqueous medium)glucose_diffusion_coeff = 6.7e-6  # cm²/slactate_diffusion_coeff = 1.0e-5  # cm²/s# ============================================================================# END OF MODIFIABLE PARAMETERS# ============================================================================# Calculate derived parameterscell_props = CELL_TYPES[cell_type]scaffold_volume = scaffold_length * scaffold_width * scaffold_height  # cm³effective_volume = scaffold_volume * scaffold_porosity  # cm³ available for flowtotal_cells = cell_density * scaffold_volume  # total cell numberchannel_width = scaffold_width  # assume flow across full widthprint(f"\n🔬 CONFIGURATION SUMMARY:")print("-" * 80)print(f"Cell type:              {cell_type}")print(f"Description:            {cell_props['description']}")print(f"Scaffold dimensions:    {scaffold_length} × {scaffold_width} × {scaffold_height} cm")print(f"Scaffold volume:        {scaffold_volume:.2f} cm³")print(f"Porosity:               {scaffold_porosity*100:.0f}%")print(f"Effective flow volume:  {effective_volume:.2f} cm³")print(f"Cell density:           {cell_density/1e6:.1f} million cells/cm³")print(f"Total cells:            {total_cells/1e6:.1f} million cells")print(f"Flow rate:              {flow_rate} μL/min")print(f"Channel height:         {channel_height*1e4:.0f} μm")# ============================================================================# SECTION 3: FLOW CHARACTERISTICS# ============================================================================print("\n" + "="*80)print("SECTION 3: FLOW CHARACTERISTICS ANALYSIS")print("="*80)# Convert flow rate to cm³/sQ_cm3_s = flow_rate / 60000  # cm³/sQ_mL_min = flow_rate / 1000  # mL/min# Calculate flow velocity (superficial velocity through porous scaffold)cross_sectional_area = scaffold_width * scaffold_height  # cm²superficial_velocity = Q_cm3_s / cross_sectional_area  # cm/sinterstitial_velocity = superficial_velocity / scaffold_porosity  # actual fluid velocity# Calculate residence timeresidence_time = effective_volume / Q_cm3_s  # secondsresidence_time_min = residence_time / 60  # minutes# Calculate Reynolds number (to verify laminar flow)hydraulic_diameter = 2 * channel_width * channel_height / (channel_width + channel_height)  # cmRe = (medium_viscosity * interstitial_velocity * hydraulic_diameter) / (medium_viscosity * 100)  # dimensionless# Note: for porous media, use modified Re calculationprint(f"\n💨 FLOW CHARACTERISTICS:")print("-" * 80)print(f"Flow rate:                    {Q_mL_min:.3f} mL/min = {Q_cm3_s*1e6:.1f} μL/s")print(f"Cross-sectional area:         {cross_sectional_area:.2f} cm²")print(f"Superficial velocity:         {superficial_velocity*10:.3f} mm/s")print(f"Interstitial velocity:        {interstitial_velocity*10:.3f} mm/s")print(f"Residence time:               {residence_time:.1f} seconds ({residence_time_min:.2f} min)")print(f"Volume exchanges per hour:    {60/residence_time_min:.1f} exchanges/hour")if residence_time < 10:    print(f"\n⚠️  SHORT RESIDENCE TIME: May not allow adequate nutrient uptake")elif residence_time > 600:    print(f"\n⚠️  LONG RESIDENCE TIME: Risk of nutrient depletion")else:    print(f"\n✅ RESIDENCE TIME: Appropriate for tissue culture")# ============================================================================# SECTION 4: WALL SHEAR STRESS CALCULATION# ============================================================================print("\n" + "="*80)print("SECTION 4: WALL SHEAR STRESS ANALYSIS")print("="*80)# Calculate wall shear stress for rectangular channel# τ = (6·μ·Q) / (w·h²)Q_m3_s = Q_cm3_s * 1e-6  # convert to m³/sw_m = channel_width / 100  # convert to mh_m = channel_height / 100  # convert to mwall_shear_stress = (6 * medium_viscosity * Q_m3_s) / (w_m * h_m**2)  # Pa# Calculate safety factorsafety_factor = cell_props['max_shear_stress'] / wall_shear_stressprint(f"\n🌊 SHEAR STRESS ANALYSIS:")print("-" * 80)print(f"Channel dimensions:           {channel_width:.2f} cm × {channel_height*1e4:.0f} μm")print(f"Medium viscosity:             {medium_viscosity} Pa·s")print(f"Calculated wall shear stress: {wall_shear_stress:.4f} Pa")print(f"Maximum tolerable shear:      {cell_props['max_shear_stress']} Pa")print(f"Safety factor:                {safety_factor:.2f}x")if wall_shear_stress <= cell_props['max_shear_stress']:    print(f"\n✅ SHEAR STRESS: ACCEPTABLE")    print(f"   {wall_shear_stress:.4f} Pa ≤ {cell_props['max_shear_stress']} Pa")    if safety_factor < 1.5:        print(f"   ⚠️  Low safety factor - consider reducing flow rate")else:    print(f"\n❌ SHEAR STRESS: TOO HIGH - RISK OF CELL DAMAGE!")    print(f"   {wall_shear_stress:.4f} Pa > {cell_props['max_shear_stress']} Pa")    max_safe_flow = (cell_props['max_shear_stress'] * w_m * h_m**2) / (6 * medium_viscosity)    max_safe_flow_uL_min = max_safe_flow * 1e9 * 60    print(f"   Maximum safe flow rate: {max_safe_flow_uL_min:.1f} μL/min")    print(f"   Or increase channel height to: {channel_height*1e4*np.sqrt(wall_shear_stress/cell_props['max_shear_stress']):.0f} μm")# ============================================================================# SECTION 5: METABOLIC CONSUMPTION RATES# ============================================================================print("\n" + "="*80)print("SECTION 5: METABOLIC CONSUMPTION & PRODUCTION RATES")print("="*80)# Calculate total consumption/production ratestotal_O2_consumption = (total_cells / 1e6) * cell_props['O2_consumption']  # nmol/stotal_glucose_consumption = (total_cells / 1e6) * cell_props['glucose_consumption']  # nmol/stotal_lactate_production = (total_cells / 1e6) * cell_props['lactate_production']  # nmol/sprint(f"\n🔬 METABOLIC RATES:")print("-" * 80)print(f"Total cells:                  {total_cells/1e6:.2f} million")print(f"Total O₂ consumption:         {total_O2_consumption:.3f} nmol/s")print(f"Total glucose consumption:    {total_glucose_consumption:.3f} nmol/s")print(f"Total lactate production:     {total_lactate_production:.3f} nmol/s")# Convert to consumption per unit volume of mediumO2_consumption_per_uL = total_O2_consumption / Q_cm3_s  # nmol/cm³glucose_consumption_per_uL = total_glucose_consumption / Q_cm3_slactate_production_per_uL = total_lactate_production / Q_cm3_sprint(f"\n📈 CONCENTRATION CHANGES (if all consumed in 1 second):")print("-" * 80)print(f"O₂ consumption rate:          {O2_consumption_per_uL:.2f} nmol/cm³/s")print(f"Glucose consumption rate:     {glucose_consumption_per_uL:.2f} nmol/cm³/s")print(f"Lactate production rate:      {lactate_production_per_uL:.2f} nmol/cm³/s")# ============================================================================# SECTION 6: CONCENTRATION GRADIENTS - SIMPLE MODEL# ============================================================================print("\n" + "="*80)print("SECTION 6: OUTLET CONCENTRATION PREDICTIONS")print("="*80)# Simple plug flow model with first-order consumption# Assumes complete mixing across scaffold and consumption along flow path# ΔC = (consumption_rate × residence_time) / volume# Calculate concentration changes over residence timeO2_depleted = (total_O2_consumption * residence_time) / (Q_cm3_s * 1000)  # μMglucose_depleted = (total_glucose_consumption * residence_time) / (Q_cm3_s * 1000)  # μMlactate_accumulated = (total_lactate_production * residence_time) / (Q_cm3_s * 1000)  # μM# Calculate outlet concentrationsoutlet_O2 = inlet_O2_concentration - O2_depletedoutlet_glucose = inlet_glucose_concentration - glucose_depletedoutlet_lactate = inlet_lactate_concentration + lactate_accumulatedprint(f"\n💧 OUTLET CONCENTRATION PREDICTIONS (Simple Model):")print("-" * 80)print(f"OXYGEN:")print(f"  Inlet:     {inlet_O2_concentration:.1f} μM")print(f"  Consumed:  {O2_depleted:.1f} μM")print(f"  Outlet:    {outlet_O2:.1f} μM")print(f"  Critical:  {critical_O2} μM")if outlet_O2 >= critical_O2:    print(f"  Status:    ✅ ADEQUATE ({outlet_O2:.1f} > {critical_O2} μM)")else:    print(f"  Status:    ❌ HYPOXIC RISK ({outlet_O2:.1f} < {critical_O2} μM)")print(f"\nGLUCOSE:")print(f"  Inlet:     {inlet_glucose_concentration:.1f} μM")print(f"  Consumed:  {glucose_depleted:.1f} μM")print(f"  Outlet:    {outlet_glucose:.1f} μM")print(f"  Critical:  {critical_glucose} μM")if outlet_glucose >= critical_glucose:    print(f"  Status:    ✅ ADEQUATE ({outlet_glucose:.1f} > {critical_glucose} μM)")else:    print(f"  Status:    ❌ STARVATION RISK ({outlet_glucose:.1f} < {critical_glucose} μM)")print(f"\nLACTATE:")print(f"  Inlet:     {inlet_lactate_concentration:.1f} μM")print(f"  Produced:  {lactate_accumulated:.1f} μM")print(f"  Outlet:    {outlet_lactate:.1f} μM")print(f"  Toxic at:  {critical_lactate} μM")if outlet_lactate <= critical_lactate:    print(f"  Status:    ✅ SAFE ({outlet_lactate:.1f} < {critical_lactate} μM)")else:    print(f"  Status:    ❌ TOXICITY RISK ({outlet_lactate:.1f} > {critical_lactate} μM)")# ============================================================================# SECTION 7: SPATIAL CONCENTRATION GRADIENTS# ============================================================================print("\n" + "="*80)print("SECTION 7: SPATIAL CONCENTRATION PROFILES")print("="*80)# Create position array along scaffold lengthn_points = 100x_positions = np.linspace(0, scaffold_length, n_points)  # cm# Calculate concentration profiles using exponential decay/accumulation# For plug flow with first-order kinetics: C(x) = C₀ × exp(-k·x/v)# where k is consumption rate constant# Calculate effective rate constants (s⁻¹)k_O2 = total_O2_consumption / (total_cells * inlet_O2_concentration * 1e-3)  # s⁻¹k_glucose = total_glucose_consumption / (total_cells * inlet_glucose_concentration * 1e-3)k_lactate = total_lactate_production / (total_cells * 1e-3)  # production, not consumption# Time at each positiontime_at_position = x_positions / interstitial_velocity  # seconds# Concentration profilesO2_profile = inlet_O2_concentration * np.exp(-k_O2 * time_at_position)glucose_profile = inlet_glucose_concentration * np.exp(-k_glucose * time_at_position)lactate_profile = inlet_lactate_concentration + (total_lactate_production/Q_cm3_s/1000) * time_at_positionprint(f"\n📍 CONCENTRATION AT KEY POSITIONS:")print("-" * 80)print(f"Position    O₂ (μM)    Glucose (μM)    Lactate (μM)")print("-" * 80)for i in [0, n_points//4, n_points//2, 3*n_points//4, -1]:    print(f"{x_positions[i]:6.2f} cm  {O2_profile[i]:8.1f}   {glucose_profile[i]:11.1f}    {lactate_profile[i]:11.1f}")# Find minimum viable regionO2_viable = O2_profile >= critical_O2glucose_viable = glucose_profile >= critical_glucoselactate_viable = lactate_profile <= critical_lactateoverall_viable = O2_viable & glucose_viable & lactate_viableif np.all(overall_viable):    print(f"\n✅ ALL POSITIONS VIABLE")    print(f"   Entire scaffold maintains adequate nutrient levels")else:    viable_fraction = np.sum(overall_viable) / len(overall_viable)    print(f"\n⚠️  PARTIAL VIABILITY")    print(f"   {viable_fraction*100:.1f}% of scaffold maintains viable conditions")    if not np.all(O2_viable):        first_hypoxic = np.where(~O2_viable)[0][0]        print(f"   Hypoxia begins at position {x_positions[first_hypoxic]:.2f} cm")# ============================================================================# SECTION 8: DAMKÖHLER NUMBER ANALYSIS# ============================================================================print("\n" + "="*80)print("SECTION 8: DIMENSIONLESS NUMBER ANALYSIS")print("="*80)# Damköhler number: Da = (reaction rate) / (diffusion rate)# Da = (k·L²) / D# where L is characteristic length scale# Effective diffusion coefficients in porous scaffoldD_eff_O2 = O2_diffusion_coeff * scaffold_porosity / scaffold_tortuosityD_eff_glucose = glucose_diffusion_coeff * scaffold_porosity / scaffold_tortuosity# Damköhler numbersDa_O2 = (k_O2 * scaffold_length**2) / D_eff_O2Da_glucose = (k_glucose * scaffold_length**2) / D_eff_glucose# Péclet number: Pe = (convection rate) / (diffusion rate)# Pe = (v·L) / DPe_O2 = (interstitial_velocity * scaffold_length) / D_eff_O2Pe_glucose = (interstitial_velocity * scaffold_length) / D_eff_glucoseprint(f"\n📐 DIMENSIONLESS NUMBERS:")print("-" * 80)print(f"Effective diffusion coefficients:")print(f"  O₂:       {D_eff_O2:.2e} cm²/s")print(f"  Glucose:  {D_eff_glucose:.2e} cm²/s")print(f"\nDamköhler number (reaction/diffusion):")print(f"  Da(O₂):      {Da_O2:.3f}")print(f"  Da(glucose): {Da_glucose:.3f}")print(f"\nPéclet number (convection/diffusion):")print(f"  Pe(O₂):      {Pe_O2:.3f}")print(f"  Pe(glucose): {Pe_glucose:.3f}")print(f"\n💡 INTERPRETATION:")print("-" * 80)if Da_O2 < 0.1:    print("Da < 0.1:  Diffusion-dominated → uniform concentration")    print("           Static culture might be sufficient")elif Da_O2 < 1:    print("0.1 < Da < 1:  Balanced regime → moderate gradients")    print("               Perfusion is beneficial")else:    print("Da > 1:  Reaction-dominated → steep gradients")    print("         Perfusion is essential!")if Pe_O2 < 1:    print("\nPe < 1:  Diffusion > Convection")    print("         Increase flow rate for better convective transport")elif Pe_O2 < 10:    print("\nPe ≈ 1-10:  Balanced convection and diffusion")    print("            Good transport regime")else:    print("\nPe > 10:  Convection >> Diffusion")    print("          Excellent perfusion, but watch shear stress")# ============================================================================# SECTION 9: OPTIMIZATION RECOMMENDATIONS# ============================================================================print("\n" + "="*80)print("SECTION 9: SYSTEM OPTIMIZATION ANALYSIS")print("="*80)# Calculate minimum flow rates for each constraintmin_flow_O2 = (total_O2_consumption / (inlet_O2_concentration - critical_O2)) * 1000 * 60  # μL/minmin_flow_glucose = (total_glucose_consumption / (inlet_glucose_concentration - critical_glucose)) * 1000 * 60max_flow_shear = (cell_props['max_shear_stress'] * w_m * h_m**2) / (6 * medium_viscosity) * 1e9 * 60  # μL/minprint(f"\n⚖️  FLOW RATE CONSTRAINTS:")print("-" * 80)print(f"Minimum for O₂ delivery:      {min_flow_O2:.1f} μL/min")print(f"Minimum for glucose delivery:  {min_flow_glucose:.1f} μL/min")print(f"Maximum for shear safety:      {max_flow_shear:.1f} μL/min")print(f"Current flow rate:             {flow_rate:.1f} μL/min")min_required = max(min_flow_O2, min_flow_glucose)operating_window_exists = min_required < max_flow_shearif operating_window_exists:    print(f"\n✅ VIABLE OPERATING WINDOW EXISTS")    print(f"   Range: {min_required:.1f} - {max_flow_shear:.1f} μL/min")        if flow_rate < min_required:        print(f"   ⚠️  Current flow TOO LOW - increase to at least {min_required:.1f} μL/min")    elif flow_rate > max_flow_shear:        print(f"   ⚠️  Current flow TOO HIGH - decrease to below {max_flow_shear:.1f} μL/min")    else:        optimal_flow = (min_required + max_flow_shear) / 2        print(f"   Current flow is within acceptable range")        print(f"   Optimal (midpoint): {optimal_flow:.1f} μL/min")else:    print(f"\n❌ NO VIABLE OPERATING WINDOW!")    print(f"   Minimum nutrient requirement ({min_required:.1f} μL/min)")    print(f"   exceeds maximum safe shear ({max_flow_shear:.1f} μL/min)")    print(f"\n   SOLUTIONS:")    print(f"   1. Reduce cell density to {cell_density * max_flow_shear / min_required / 1e6:.1f} M cells/cm³")    print(f"   2. Increase channel height to {channel_height*1e4*np.sqrt(min_required/max_flow_shear):.0f} μm")    print(f"   3. Use multiple parallel perfusion channels")    print(f"   4. Implement indirect perfusion (flow around construct)")# ============================================================================# SECTION 10: COMPREHENSIVE VISUALIZATION# ============================================================================print("\n" + "="*80)print("SECTION 10: VISUALIZATION OF CONCENTRATION PROFILES")print("="*80)fig, axes = plt.subplots(2, 2, figsize=(14, 10))fig.suptitle(f'Perfusion System Analysis: {cell_type}\n' +              f'Flow Rate: {flow_rate} μL/min, Cell Density: {cell_density/1e6:.1f} M cells/cm³',             fontsize=14, fontweight='bold')# Plot 1: Oxygen gradientax1 = axes[0, 0]ax1.plot(x_positions*10, O2_profile, 'b-', linewidth=2.5, label='O₂ concentration')ax1.axhline(y=critical_O2, color='r', linestyle='--', linewidth=2, label=f'Critical O₂ ({critical_O2} μM)')ax1.fill_between(x_positions*10, critical_O2, O2_profile,                   where=(O2_profile >= critical_O2), alpha=0.3, color='green', label='Viable zone')ax1.fill_between(x_positions*10, 0, critical_O2,                  where=(O2_profile < critical_O2), alpha=0.3, color='red', label='Hypoxic zone')ax1.set_xlabel('Position Along Flow Direction (mm)', fontsize=11)ax1.set_ylabel('O₂ Concentration (μM)', fontsize=11)ax1.set_title('Oxygen Concentration Profile', fontsize=12, fontweight='bold')ax1.legend(loc='best')ax1.grid(True, alpha=0.3)ax1.set_ylim(bottom=0)# Plot 2: Glucose gradientax2 = axes[0, 1]ax2.plot(x_positions*10, glucose_profile, 'g-', linewidth=2.5, label='Glucose concentration')ax2.axhline(y=critical_glucose, color='r', linestyle='--', linewidth=2, label=f'Critical glucose ({critical_glucose} μM)')ax2.fill_between(x_positions*10, critical_glucose, glucose_profile,                  where=(glucose_profile >= critical_glucose), alpha=0.3, color='green', label='Viable zone')ax2.set_xlabel('Position Along Flow Direction (mm)', fontsize=11)ax2.set_ylabel('Glucose Concentration (μM)', fontsize=11)ax2.set_title('Glucose Concentration Profile', fontsize=12, fontweight='bold')ax2.legend(loc='best')ax2.grid(True, alpha=0.3)ax2.set_ylim(bottom=0)# Plot 3: Lactate accumulationax3 = axes[1, 0]ax3.plot(x_positions*10, lactate_profile, 'm-', linewidth=2.5, label='Lactate concentration')ax3.axhline(y=critical_lactate, color='r', linestyle='--', linewidth=2, label=f'Toxic level ({critical_lactate} μM)')ax3.fill_between(x_positions*10, 0, lactate_profile,                  where=(lactate_profile <= critical_lactate), alpha=0.3, color='green', label='Safe zone')ax3.set_xlabel('Position Along Flow Direction (mm)', fontsize=11)ax3.set_ylabel('Lactate Concentration (μM)', fontsize=11)ax3.set_title('Lactate Accumulation Profile', fontsize=12, fontweight='bold')ax3.legend(loc='best')ax3.grid(True, alpha=0.3)ax3.set_ylim(bottom=0)# Plot 4: Flow rate constraintsax4 = axes[1, 1]flow_range = np.linspace(1, max(max_flow_shear * 1.5, flow_rate * 1.5), 100)shear_values = [(6 * medium_viscosity * (fr/60000/1e6)) / (w_m * h_m**2) for fr in flow_range]ax4.plot(flow_range, shear_values, 'b-', linewidth=2, label='Shear stress')ax4.axhline(y=cell_props['max_shear_stress'], color='r', linestyle='--', linewidth=2,            label=f'Max tolerable ({cell_props["max_shear_stress"]} Pa)')ax4.axvline(x=min_flow_O2, color='orange', linestyle=':', linewidth=2, label=f'Min for O₂ ({min_flow_O2:.0f} μL/min)')ax4.axvline(x=min_flow_glucose, color='purple', linestyle=':', linewidth=2, label=f'Min for glucose ({min_flow_glucose:.0f} μL/min)')ax4.axvline(x=flow_rate, color='green', linestyle='-', linewidth=2.5, label=f'Current ({flow_rate} μL/min)')# Shade viable regionif operating_window_exists:    ax4.axvspan(min_required, max_flow_shear, alpha=0.2, color='green', label='Operating window')ax4.set_xlabel('Flow Rate (μL/min)', fontsize=11)ax4.set_ylabel('Wall Shear Stress (Pa)', fontsize=11)ax4.set_title('Flow Rate Optimization', fontsize=12, fontweight='bold')ax4.legend(loc='best', fontsize=8)ax4.grid(True, alpha=0.3)ax4.set_ylim(bottom=0)plt.tight_layout()plt.show()print("\n✅ Visualization complete!")# ============================================================================# SECTION 11: SUMMARY TABLE# ============================================================================print("\n" + "="*80)print("SECTION 11: PERFORMANCE SUMMARY")print("="*80)print(f"\n📊 SYSTEM PERFORMANCE METRICS:")print("-" * 80)print(f"Parameter                          Value              Status")print("-" * 80)print(f"Flow rate                          {flow_rate:8.1f} μL/min    {('✅' if min_required <= flow_rate <= max_flow_shear else '❌')}")print(f"Residence time                     {residence_time:8.1f} s         {('✅' if 10 <= residence_time <= 600 else '⚠️ ')}")print(f"Wall shear stress                  {wall_shear_stress:8.4f} Pa        {('✅' if wall_shear_stress <= cell_props['max_shear_stress'] else '❌')}")print(f"Outlet O₂                          {outlet_O2:8.1f} μM        {('✅' if outlet_O2 >= critical_O2 else '❌')}")print(f"Outlet glucose                     {outlet_glucose:8.1f} μM        {('✅' if outlet_glucose >= critical_glucose else '❌')}")print(f"Outlet lactate                     {outlet_lactate:8.1f} μM        {('✅' if outlet_lactate <= critical_lactate else '❌')}")print(f"Damköhler number                   {Da_O2:8.3f}           {('Perfusion essential' if Da_O2 > 1 else 'Moderate gradients')}")print(f"Péclet number                      {Pe_O2:8.3f}           {('Convection-dominated' if Pe_O2 > 10 else 'Balanced transport')}")overall_status = (min_required <= flow_rate <= max_flow_shear and                  outlet_O2 >= critical_O2 and                  outlet_glucose >= critical_glucose and                  outlet_lactate <= critical_lactate)print("-" * 80)if overall_status:    print("OVERALL ASSESSMENT: ✅ SYSTEM VIABLE FOR LONG-TERM CULTURE")else:    print("OVERALL ASSESSMENT: ❌ SYSTEM REQUIRES OPTIMIZATION")# ============================================================================# SECTION 12: TASKS FOR STUDENTS# ============================================================================print("\n" + "="*80)print("SECTION 12: TASKS & QUESTIONS")print("="*80)print("""Complete the following tasks by modifying parameters in SECTION 2 and re-running:TASK 1: BASELINE CHARACTERIZATION (Current Configuration)   Record the following for Hepatocytes at 100 μL/min:   a) What is the outlet O₂ concentration? Is it above the critical threshold?   b) What is the wall shear stress? Is it safe for the cells?   c) What is the Damköhler number? What does this tell you about the transport regime?   d) Is there a viable operating window? If yes, what is the range?TASK 2: FLOW RATE OPTIMIZATION   Keep Hepatocytes but try different flow rates:   a) Decrease flow to 25 μL/min - what happens to O₂ and shear stress?   b) Increase flow to 400 μL/min - what happens to O₂ and shear stress?   c) Find the optimal flow rate that balances nutrient delivery with shear safety   d) What is the minimum flow rate needed to prevent hypoxia?TASK 3: CELL TYPE COMPARISON   Compare three cell types at the same flow rate (100 μL/min):   a) Hepatocytes (baseline)   b) Cardiomyocytes (high metabolism)   c) Chondrocytes (low metabolism, shear-sensitive)      For each cell type, record:   - Outlet O₂ concentration   - Is shear stress acceptable?   - Operating window range (if exists)   - Which cell type is easiest to culture in perfusion? Why?TASK 4: SCALING ANALYSIS   Start with Hepatocytes at 100 μL/min, then:   a) DOUBLE the cell density (20e6 cells/cm³) - what happens?   b) Return to 10e6 cells/cm³ but DOUBLE the scaffold length (2.0 cm)   c) Which scaling approach (density vs. size) is more problematic? Why?   d) For the doubled length scaffold, what flow rate is needed to maintain viability?TASK 5: SHEAR-SENSITIVE CELLS   Configure for Chondrocytes (very shear-sensitive):   a) Start with default settings - is shear stress acceptable?   b) What is the maximum safe flow rate for chondrocytes?   c) At this maximum safe flow, is O₂ delivery adequate?   d) If not viable, propose TWO solutions (hint: adjust channel height or cell density)TASK 6: HIGH-DEMAND TISSUE   Configure for Cardiomyocytes (high metabolic demand):   a) Use default 100 μL/min - is O₂ delivery sufficient?   b) What minimum flow rate is needed for adequate O₂ delivery?   c) At this flow rate, is shear stress still safe?   d) Calculate the optimal flow rate for this tissue typeTASK 7: CHANNEL GEOMETRY OPTIMIZATION   For Hepatocytes at 100 μL/min with high shear stress problem:   a) What happens when you DOUBLE the channel height (200e-4 cm)?   b) What happens when you HALVE the channel height (50e-4 cm)?   c) Explain the relationship between channel height and shear stress   d) Is there a downside to making channels too tall?TASK 8: POROUS SCAFFOLD EFFECTS   For Hepatocytes at 100 μL/min:   a) Decrease porosity to 0.4 (40%) - how does this affect residence time?   b) Increase porosity to 0.9 (90%) - how does this affect residence time?   c) What porosity value gives the best balance? Why?   d) How does tortuosity (currently 2.0) affect diffusion?TASK 9: LONG-TERM CULTURE PLANNING   Design a 14-day perfusion culture for Hepatocytes:   a) Calculate total medium consumption over 14 days at 100 μL/min   b) If you want to minimize medium use, what's the minimum viable flow rate?   c) Calculate daily glucose consumption (g/day) at this flow rate   d) Would you recommend continuous or intermittent perfusion? Why?TASK 10: MULTI-PARAMETER OPTIMIZATION   Design an optimal perfusion system for MSCs:   a) Choose appropriate cell density   b) Calculate optimal flow rate   c) Select channel height for safe shear stress   d) Verify all parameters (O₂, glucose, lactate, shear) are within acceptable ranges   e) Calculate the Damköhler and Péclet numbers - are they in good regimes?REFLECTION QUESTIONS:1. Why is perfusion superior to static culture for thick constructs?2. What is the fundamental trade-off in perfusion system design?3. How does the Damköhler number help predict whether perfusion is necessary?4. Why do different cell types require different perfusion strategies?5. In a real bioreactor, what sensors would you install and where?WRITE YOUR ANSWERS HERE:________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________""")print("\n" + "="*80)print("🎉 EXERCISE 5 COMPLETE!")print("="*80)print("""KEY TAKEAWAYS:✓ Perfusion balances nutrient delivery with mechanical stress✓ Different cell types have vastly different metabolic and mechanical requirements✓ Damköhler number predicts transport regime (diffusion vs. reaction-limited)✓ Flow rate optimization requires considering multiple constraints simultaneously✓ Channel geometry strongly affects shear stress✓ Scaffold properties (porosity, tortuosity) affect both flow and diffusionNEXT STEPS:→ Complete all tasks by modifying parameters in Section 2→ Record your observations for each configuration→ Compare results across different cell types→ Proceed to Exercise 6: Bioreactor Scale-Up Economics""")print("\n📚 Ready to optimize perfusion systems!")print("="*80)